<html>
    <div style="padding: 50px; background-color: lightskyblue; text-align: center; color: #000;">
        <h1>CAB420 Workshop - Week 1 Practical</h1>
        <h2>Linear Regression</h2>
    </div>
</html>

Using the provided dataset (either the individual files and your own approach to merge the data, or the `combined.csv` merged dataset), split the data into training, validation and testing as follows:

- Training: All data from the years 2014-2018
- Validation: All data from 2017
- Testing: All data from 2018

Develop a regression model to predict one of the cycleway data series (select whichever one takes your fancy) in your dataset. In developing the model, you should:

- Initially, use all weather data (temperature, rainfall, solar exposure) and all other data series for a particular counter type (i.e. if you're predicting cyclists inbound for the counter, use all other cyclist inbound counters).

- Use p-values, qqplot, correlation between predictors andresponse, correlation between pairs of predictor, sand performance on the validation set to remove terms and improve the model

When you have finished refining the model, evaluate it on the test set and compare the Root Mean Squared Error (RMSE) for the training, validation and test sets. 

In training the model, you will need to ensure that you have no samples (i.e.rows) with missing data. As such, you should remove samples with missing data from the dataset before training and evaluating the model. This may also mean that you have to remove some columns that contain large amounts of missing data (i.e. determine how many samples are missing from each column, remove columns with lots of missing data, remove any other rows where data is missing). 

We recommend the `statsmodels` and `pandas` packages for this problem. In particular, you may wish to use:
- `isna`: A member function of a pandas dataframe that indicates if a variable is missing 
- `dropna`: A member function of a pandas dataframe that drops missing values
- `statsmodels.api.OLS`: Ordinary Least Squares regression function within `statsmodels`.

You may also wish to explore the `sklearn` package which also contains methods for linear regression and data splitting.

## Installing Packages Loading in the Dataset

In [74]:
import pandas as pd
from  sklearn.linear_model import LinearRegression
import statsmodels.api as sm

In [75]:
# Cyclist counts
counts2014 = pd.read_csv('data/bike-ped-auto-counts-2014.csv')
counts2015 = pd.read_csv('data/bike-ped-auto-counts-2015.csv')
counts2016 = pd.read_csv('data/bike-ped-auto-counts-2016.csv')
counts2017 = pd.read_csv('data/bike-ped-auto-counts-2017.csv')
counts2018 = pd.read_csv('data/bike-ped-auto-counts-2018.csv')

# BOM weather data
rainfall_data = pd.read_csv('data/IDCJAC0009_040913_1800_Data.csv')
temperature_data = pd.read_csv('data/IDCJAC0010_040913_1800_Data.csv')
solar_exposure_data = pd.read_csv('data/IDCJAC0016_040913_1800_Data.csv')

# Slicing the data by year
# Rainfall
rainfall_data_2014 = rainfall_data[rainfall_data['Year'] == 2014]
rainfall_data_2015 = rainfall_data[rainfall_data['Year'] == 2015]
rainfall_data_2016 = rainfall_data[rainfall_data['Year'] == 2016]
rainfall_data_2017 = rainfall_data[rainfall_data['Year'] == 2017]
rainfall_data_2018 = rainfall_data[rainfall_data['Year'] == 2018]

# Temperature
temperature_data_2014 = temperature_data[temperature_data['Year'] == 2014]
temperature_data_2015 = temperature_data[temperature_data['Year'] == 2015]
temperature_data_2016 = temperature_data[temperature_data['Year'] == 2016]
temperature_data_2017 = temperature_data[temperature_data['Year'] == 2017]
temperature_data_2018 = temperature_data[temperature_data['Year'] == 2018]

# Solar exposure
solar_exposure_data_2014 = solar_exposure_data[solar_exposure_data['Year'] == 2014]
solar_exposure_data_2015 = solar_exposure_data[solar_exposure_data['Year'] == 2015]
solar_exposure_data_2016 = solar_exposure_data[solar_exposure_data['Year'] == 2016]
solar_exposure_data_2017 = solar_exposure_data[solar_exposure_data['Year'] == 2017]
solar_exposure_data_2018 = solar_exposure_data[solar_exposure_data['Year'] == 2018]

In [76]:
counts2014

,Date,Unnamed: 1,Bicenntenial Bikeway,Bicentennial Bikeway Pedestrians Inbound,Bicentennial Bikeway Pedestrians Outbound,Bicentennial Bikeway Cyclists Inbound,Bicentennial Bikeway Cyclists Outbound,Bishop Street Enoggera,Bishop Street Pedestrians Inbound,Bishop Street Pedestrians Outbound,...,Story Bridge East,Story Bridge East Pedestrian Inbound,Story Bridge East Pedestrian Outbound,Story Bridge East Cyclists Inbound,Story Bridge East Cyclists Outbound,Story Bridge West,Story Bridge West Pedestrian Inbound,Story Bridge West Pedestrian Outbound,Story Bridge West Cyclists Inbound,Story Bridge West Cyclists Outbound
0,1/01/2014,0:00,3333,866,1326,569,572,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
1,2/01/2014,0:00,4863,1372,2133,658,700,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2,3/01/2014,0:00,3905,1163,1867,463,412,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,4/01/2014,0:00,3066,770,1522,410,364,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,5/01/2014,0:00,4550,1153,2102,654,641,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,27/12/2014,0:00,95,0,0,46,49,NaN,NaN,NaN,...,0,0,0,0,0,313,170,110,26,7
361,28/12/2014,0:00,188,0,0,92,96,NaN,NaN,NaN,...,0,0,0,0,0,731,372,235,94,30
362,29/12/2014,0:00,287,0,0,161,126,NaN,NaN,NaN,...,0,0,0,0,0,697,255,191,180,71
363,30/12/2014,0:00,451,0,0,304,147,NaN,NaN,NaN,...,0,0,0,0,0,728,295,162,216,55


Looks like there are a few columns with missing data. Time to check which ones they are and remove them accordingly

In [77]:
for variable in counts2014.columns.to_list():
    print(f'Number of missing values for {variable}: {counts2014[variable].isna().sum()}')

Number of missing values for Date: 0
Number of missing values for Unnamed: 1: 0
Number of missing values for Bicenntenial Bikeway: 0
Number of missing values for Bicentennial Bikeway Pedestrians Inbound: 0
Number of missing values for Bicentennial Bikeway Pedestrians Outbound: 0
Number of missing values for Bicentennial Bikeway Cyclists Inbound: 0
Number of missing values for Bicentennial Bikeway Cyclists Outbound: 0
Number of missing values for Bishop Street Enoggera: 365
Number of missing values for Bishop Street Pedestrians Inbound: 365
Number of missing values for Bishop Street Pedestrians Outbound: 365
Number of missing values for Bishop Street Cyclists Inbound: 365
Number of missing values for Bishop Street Cyclists Outbound: 365
Number of missing values for Ekibin Park: 365
Number of missing values for Ekibin Park Pedestrians Inbound: 365
Number of missing values for Ekibin Park Pedestrians Outbound: 365
Number of missing values for Ekibin Park Cyclists Inbound: 365
Number of mi

There are many counts that do not have a single record. Removing those from the dataset

In [ ]:
for variable in counts2014.columns.to_list():
    if counts2014[variable].isna().sum() > 5:
        counts2014.drop(variable, axis=1, inplace=True)

for variable in counts2014.columns.to_list():
    if 'Pedestrians' in variable:
        counts2014.drop(variable, axis=1, inplace=True)
counts2014.drop('Unnamed: 1', axis=1, inplace=True)

KeyError: 'Bicentennial Bikeway Pedestrians Inbound'

In [ ]:
counts2014

,Date,Bicenntenial Bikeway,Bicentennial Bikeway Pedestrians Inbound,Bicentennial Bikeway Pedestrians Outbound,Bicentennial Bikeway Cyclists Inbound,Bicentennial Bikeway Cyclists Outbound,Eleanor Schonell Bridge Cyclists,Eleanor Schonell Bridge Cyclists Inbound,Eleanor Schonell Bridge Cyclists Outbound,Eleanor Schonell Bridge Pedestrians,...,Story Bridge East,Story Bridge East Pedestrian Inbound,Story Bridge East Pedestrian Outbound,Story Bridge East Cyclists Inbound,Story Bridge East Cyclists Outbound,Story Bridge West,Story Bridge West Pedestrian Inbound,Story Bridge West Pedestrian Outbound,Story Bridge West Cyclists Inbound,Story Bridge West Cyclists Outbound
0,1/01/2014,3333,866,1326,569,572,691,301,390,474,...,0,0,0,0,0,0,0,0,0,0
1,2/01/2014,4863,1372,2133,658,700,1020,473,547,798,...,0,0,0,0,0,0,0,0,0,0
2,3/01/2014,3905,1163,1867,463,412,931,442,489,635,...,0,0,0,0,0,0,0,0,0,0
3,4/01/2014,3066,770,1522,410,364,574,245,329,387,...,0,0,0,0,0,0,0,0,0,0
4,5/01/2014,4550,1153,2102,654,641,941,398,543,570,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,27/12/2014,95,0,0,46,49,511,511,0,137,...,0,0,0,0,0,313,170,110,26,7
361,28/12/2014,188,0,0,92,96,0,0,0,361,...,0,0,0,0,0,731,372,235,94,30
362,29/12/2014,287,0,0,161,126,0,0,0,454,...,0,0,0,0,0,697,255,191,180,71
363,30/12/2014,451,0,0,304,147,0,0,0,385,...,0,0,0,0,0,728,295,162,216,55


In [ ]:
for variable in counts2014.columns.to_list():
    print(f'Number of missing values for {variable}: {counts2014[variable].isna().sum()}')

Number of missing values for Date: 0
Number of missing values for Bicenntenial Bikeway: 0
Number of missing values for Bicentennial Bikeway Pedestrians Inbound: 0
Number of missing values for Bicentennial Bikeway Pedestrians Outbound: 0
Number of missing values for Bicentennial Bikeway Cyclists Inbound: 0
Number of missing values for Bicentennial Bikeway Cyclists Outbound: 0
Number of missing values for Eleanor Schonell Bridge Cyclists: 0
Number of missing values for Eleanor Schonell Bridge Cyclists Inbound: 0
Number of missing values for Eleanor Schonell Bridge Cyclists Outbound: 0
Number of missing values for Eleanor Schonell Bridge Pedestrians: 0
Number of missing values for Eleanor Schonell Bridge Pedestrians Inbound: 0
Number of missing values for Eleanor Schonell Bridge Pedestrians Outbound: 0
Number of missing values for Schulz Canal Bridge: 0
Number of missing values for Schulz Canal Bridge Pedestrians Inbound: 0
Number of missing values for Schulz Canal Bridge Pedestrians Outb

Now that the data has been cleaned of na values, time to build the initial Linear Regression model.

Dropping the date column from the regression dataset

In [ ]:
X_2014 = counts2014.drop('Date', axis=1)

In [ ]:
X_2014

,Bicenntenial Bikeway,Bicentennial Bikeway Pedestrians Inbound,Bicentennial Bikeway Pedestrians Outbound,Bicentennial Bikeway Cyclists Inbound,Bicentennial Bikeway Cyclists Outbound,Eleanor Schonell Bridge Cyclists,Eleanor Schonell Bridge Cyclists Inbound,Eleanor Schonell Bridge Cyclists Outbound,Eleanor Schonell Bridge Pedestrians,Eleanor Schonell Bridge Pedestrians Inbound,...,Story Bridge East,Story Bridge East Pedestrian Inbound,Story Bridge East Pedestrian Outbound,Story Bridge East Cyclists Inbound,Story Bridge East Cyclists Outbound,Story Bridge West,Story Bridge West Pedestrian Inbound,Story Bridge West Pedestrian Outbound,Story Bridge West Cyclists Inbound,Story Bridge West Cyclists Outbound
0,3333,866,1326,569,572,691,301,390,474,248,...,0,0,0,0,0,0,0,0,0,0
1,4863,1372,2133,658,700,1020,473,547,798,413,...,0,0,0,0,0,0,0,0,0,0
2,3905,1163,1867,463,412,931,442,489,635,336,...,0,0,0,0,0,0,0,0,0,0
3,3066,770,1522,410,364,574,245,329,387,196,...,0,0,0,0,0,0,0,0,0,0
4,4550,1153,2102,654,641,941,398,543,570,300,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,95,0,0,46,49,511,511,0,137,58,...,0,0,0,0,0,313,170,110,26,7
361,188,0,0,92,96,0,0,0,361,179,...,0,0,0,0,0,731,372,235,94,30
362,287,0,0,161,126,0,0,0,454,232,...,0,0,0,0,0,697,255,191,180,71
363,451,0,0,304,147,0,0,0,385,203,...,0,0,0,0,0,728,295,162,216,55


Using statsmodels:

In [ ]:
statsmodels_lm = sm.OLS(y, X).fit()
statsmodels_lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                  OLS Regression Results                                 
=========================================================================================
Dep. Variable:     Bicenntenial Bikeway   R-squared (uncentered):                   1.000
Model:                              OLS   Adj. R-squared (uncentered):              1.000
Method:                   Least Squares   F-statistic:                          1.535e+31
Date:                  Thu, 27 Feb 2025   Prob (F-statistic):                        0.00
Time:                          15:02:08   Log-Likelihood:                          8936.3
No. Observations:                   365   AIC:                                 -1.784e+04
Df Residuals:                       347   BIC:                                 -1.777e+04
Df Model:                            18                                                  
Covariance Type:              nonrobust                                                  
================================================================================================================
                                                   coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------
Bicenntenial Bikeway                             0.8000    3.5e-16   2.29e+15      0.000       0.800       0.800
Bicentennial Bikeway Pedestrians Inbound         0.2000   1.53e-15   1.31e+14      0.000       0.200       0.200
Bicentennial Bikeway Pedestrians Outbound        0.2000   1.49e-15   1.34e+14      0.000       0.200       0.200
Bicentennial Bikeway Cyclists Inbound            0.2000   3.68e-15   5.44e+13      0.000       0.200       0.200
Bicentennial Bikeway Cyclists Outbound           0.2000   3.78e-15    5.3e+13      0.000       0.200       0.200
Eleanor Schonell Bridge Cyclists              7.355e-16   9.27e-16      0.793      0.428   -1.09e-15    2.56e-15
Eleanor Schonell Bridge Cyclists Inbound      2.442e-15   5.87e-15      0.416      0.677    -9.1e-15     1.4e-14
Eleanor Schonell Bridge Cyclists Outbound    -3.331e-16   5.86e-15     -0.057      0.955   -1.19e-14    1.12e-14
Eleanor Schonell Bridge Pedestrians            1.36e-15   5.94e-16      2.290      0.023    1.92e-16    2.53e-15
Eleanor Schonell Bridge Pedestrians Inbound   3.331e-16   1.02e-15      0.325      0.745   -1.68e-15    2.35e-15
Eleanor Schonell Bridge Pedestrians Outbound   1.11e-15   1.44e-15      0.769      0.443   -1.73e-15    3.95e-15
Schulz Canal Bridge                           3.997e-15   6.82e-15      0.586      0.558   -9.41e-15    1.74e-14
Schulz Canal Bridge Pedestrians Inbound       4.885e-15   1.84e-14      0.266      0.790   -3.12e-14     4.1e-14
Schulz Canal Bridge Pedestrians Outbound              0   3.89e-14          0      1.000   -7.66e-14    7.66e-14
Schulz Canal Bridge Cyclists Inbound          2.309e-14   3.52e-14      0.656      0.513   -4.62e-14    9.24e-14
Schulz Canal Bridge Cyclists Outbound        -1.954e-14    3.7e-14     -0.528      0.598   -9.23e-14    5.33e-14
Story Bridge East                             5.329e-15   4.18e-15      1.276      0.203   -2.89e-15    1.35e-14
Story Bridge East Pedestrian Inbound          -1.51e-14   2.17e-14     -0.695      0.487   -5.78e-14    2.76e-14
Story Bridge East Pedestrian Outbound          9.77e-15   2.53e-14      0.386      0.700   -4.01e-14    5.96e-14
Story Bridge East Cyclists Inbound                    0          0        nan        nan           0           0
Story Bridge East Cyclists Outbound                   0          0        nan        nan           0           0
Story Bridge West                            -1.388e-15   1.11e-15     -1.250      0.212   -3.57e-15    7.96e-16
Story Bridge West Pedestrian Inbound         -7.216e-16   4.36e-15     -0.166      0.869   -9.29e-15    7.85e-15
Story Bridge West Pedest